# 창폐업 데이터 확인

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 한글폰트 사용 in colab
%matplotlib inline  

import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

NanumBarunGothic


In [3]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import os
import time
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

In [4]:
os.listdir("/content/drive/MyDrive/KGU/공모전/KT_빅데이터플랫폼_데이터셋/")

['전체 데이터 설명.hwp',
 'popl.csv',
 'dlvr_call.csv',
 'StartupOrClosing_Region_ver2.csv',
 'StartupOrClosing_Shop_ver2.csv']

In [5]:
region = pd.read_csv("/content/drive/MyDrive/KGU/공모전/KT_빅데이터플랫폼_데이터셋/StartupOrClosing_Region_ver2.csv")
shop = pd.read_csv("/content/drive/MyDrive/KGU/공모전/KT_빅데이터플랫폼_데이터셋/StartupOrClosing_Shop_ver2.csv")
display(region.head())
display(shop.head())

,INDEX,DATE,INDUTY_NM,BRTC_NM,SIGUNGU_NM,EMD,LA_DCMLPOINT_VALUE,LO_DCMLPOINT_VALUE,DIVISION
0,1,20200101,부동산,경기,용인시 처인구,봉명리,37.109***,127.131***,1
1,2,20200101,토목공학엔지니어링,인천,서구,오류동,37.598***,126.613***,1
2,3,20200101,한식,강원,영월군 영월읍,영흥리,37.187***,128.473***,1
3,4,20200101,보일러판매,대전,중구,목동,36.336***,127.403***,1
4,5,20200101,돼지국밥,경남,창원시 마산합포구,현동,35.147***,128.557***,1


,No,DATE,INDUTY_NM,CMPNM,INDUTY_CODEM,division
0,1,20200101,CCTV,(주)실라전자,319013,1
1,2,20200101,가공식품도매(기타),송현동1986발효이야기,452199,1
2,3,20200101,가구판매(종합),더갤러리전주점,571100,1
3,4,20200101,가정부.파출부소개,모두인력파출,839103,1
4,5,20200101,가정부업알선,모두인력파출,839101,1


In [7]:
# 날짜가 폐업 날짜가 아닌 데이터 생성날짜
# 어떻게 활용할 수 있을지? ==> 업종별로 파악하는 수밖에 없음
# 딱히 일자를 맞춰서 쓰기 보단, 총합을 계산해야 할 듯 하다.
# 1월 20일 전은 데이터 생성 시기로 따져봐도 코로나 전으로 판단
# 1월 20일 후부터 코로나 영향권으로 볼 지에 대한 의문
# 배달 호출데이터는 3월 31일까지 이므로, 그 후 데이터 사용?


# 날짜에 대해선 팀원과 논의 후 결정

print(region['DATE'].unique())
print(shop['DATE'].unique())

[20200101 20200102 20200103 20200104 20200105 20200106 20200107 20200108
 20200109 20200110 20200111 20200113 20200114 20200115 20200117 20200118
 20200119 20200120 20200121 20200122 20200123 20200124 20200125 20200128
 20200129 20200130 20200131 20200201 20200202 20200203 20200204 20200205
 20200206 20200207 20200208 20200209 20200210 20200211 20200212 20200213
 20200214 20200215 20200216 20200217 20200218 20200219 20200220 20200221
 20200222 20200223 20200224 20200225 20200227 20200229 20200301 20200302
 20200303 20200304 20200305 20200306 20200307 20200308 20200309 20200310
 20200311 20200312 20200313 20200314 20200315 20200316 20200317 20200318
 20200319 20200320 20200321 20200322 20200324 20200325 20200327 20200330
 20200331 20200401 20200402 20200403 20200404 20200405 20200406 20200408
 20200409 20200410 20200411 20200412 20200413 20200414 20200415 20200416
 20200417 20200418 20200419 20200420 20200421 20200422 20200423 20200424
 20200425 20200426 20200427 20200506 20200508 20200

In [8]:
# 위도 경도로 특정 구에 얼마나 폐업했는지는 파악 가능
# 다만 지역을 서울, 경기로 한정 ==> 배달데이터가 서울 경기
sudo = region[(region['BRTC_NM']=='서울') | (region['BRTC_NM']=='경기')]
sudo

,INDEX,DATE,INDUTY_NM,BRTC_NM,SIGUNGU_NM,EMD,LA_DCMLPOINT_VALUE,LO_DCMLPOINT_VALUE,DIVISION
0,1,20200101,부동산,경기,용인시 처인구,봉명리,37.109***,127.131***,1
6,7,20200102,햄버거,경기,용인시 처인구,완장리,37.156***,127.173***,1
7,8,20200102,악기판매,경기,안양시 만안구,안양1동,37.407***,126.914***,1
15,16,20200102,소프트웨어개발(기타),서울,관악구,조원[신림8동]동,37.484***,126.903***,0
18,19,20200102,산후조리원,서울,서초구,반포동,37.502***,127.010***,0
...,...,...,...,...,...,...,...,...,...
67247,67248,20200630,한식,서울,성동구,행당동,37.559***,127.040***,0
67251,67252,20200630,프랜차이즈,서울,강남구,개포동,37.478***,127.047***,0
67253,67254,20200630,호프,경기,용인시 기흥구,공세동,37.240***,127.108***,0
67254,67255,20200630,떡볶이,경기,성남시 분당구,야탑동,37.412***,127.129***,0


In [9]:
# 날짜 상관없이 창업 폐업만 알아볼까?
sudo['DIVISION'].value_counts() # 폐업이 압도적으로 많다. ==> 나중에 시간순으로도 보면 좋을 듯

0    23193
1      686
Name: DIVISION, dtype: int64

In [12]:
# 서울 내 특정 구 확인을 위한 전처리
# replace가 안먹히네
# 그냥 슬라이싱으로 처리

for x in tqdm(range(len(sudo))) :
  sudo['LA_DCMLPOINT_VALUE'].iloc[x] = sudo['LA_DCMLPOINT_VALUE'].iloc[x][:-3]
  sudo['LO_DCMLPOINT_VALUE'].iloc[x] = sudo['LO_DCMLPOINT_VALUE'].iloc[x][:-3]

sudo.head()

100%|██████████| 23879/23879 [00:15<00:00, 1527.81it/s]


,INDEX,DATE,INDUTY_NM,BRTC_NM,SIGUNGU_NM,EMD,LA_DCMLPOINT_VALUE,LO_DCMLPOINT_VALUE,DIVISION
0,1,20200101,부동산,경기,용인시 처인구,봉명리,37.109,127.131,1
6,7,20200102,햄버거,경기,용인시 처인구,완장리,37.156,127.173,1
7,8,20200102,악기판매,경기,안양시 만안구,안양1동,37.407,126.914,1
15,16,20200102,소프트웨어개발(기타),서울,관악구,조원[신림8동]동,37.484,126.903,0
18,19,20200102,산후조리원,서울,서초구,반포동,37.502,127.010,0


In [13]:
sudo['ID'] = sudo['LA_DCMLPOINT_VALUE'] + str("::") + sudo['LO_DCMLPOINT_VALUE']
sudo.head()

,INDEX,DATE,INDUTY_NM,BRTC_NM,SIGUNGU_NM,EMD,LA_DCMLPOINT_VALUE,LO_DCMLPOINT_VALUE,DIVISION,ID
0,1,20200101,부동산,경기,용인시 처인구,봉명리,37.109,127.131,1,37.109::127.131
6,7,20200102,햄버거,경기,용인시 처인구,완장리,37.156,127.173,1,37.156::127.173
7,8,20200102,악기판매,경기,안양시 만안구,안양1동,37.407,126.914,1,37.407::126.914
15,16,20200102,소프트웨어개발(기타),서울,관악구,조원[신림8동]동,37.484,126.903,0,37.484::126.903
18,19,20200102,산후조리원,서울,서초구,반포동,37.502,127.010,0,37.502::127.010


In [14]:
sudo.groupby(by='ID', as_index=False).size() # 알아보기 위해서는 점을 찍고 마커클러스터로 수를 확인하는 것이 가장 효율적일 듯 하다.
## 아니면 뒤에 한자리씩 자르고 size 수를 늘려도 될 것 같긴 하다.
## 클러스터와 공시지가를 합쳐서 시각화하는 것도 가능하다.


,ID,size
0,34.889::128.606,2
1,34.957::128.414,1
2,35.082::128.903,1
3,35.105::128.828,1
4,35.177::129.073,2
...,...,...
17056,38.142::127.062,1
17057,38.153::127.143,1
17058,38.154::127.248,1
17059,38.157::127.249,1


In [4]:
######################################################### 번외편 주문지역 인구특성

In [15]:
pop = pd.read_csv("/content/drive/MyDrive/KGU/공모전/KT_빅데이터플랫폼_데이터셋/popl.csv")

In [16]:
display(pop.head())

,base_ymd,ori_table,tmzon_se_code,sexdstn_se_code,agrde_se_code,adstrd_code,local_pop,fore_pop_1,fore_pop_2
0,20200212,2,F,age_50,11110515,205,0,0,NaN
1,20200212,2,F,age_15,11110515,78,0,0,NaN
2,20200212,21,F,age_10,11110515,28,0,0,NaN
3,20200212,7,F,age_10,11110515,49,0,0,NaN
4,20200212,16,M,age_25,11110515,291,0,0,NaN
